# Amazon Web Scrapper

Requirements:

1. Selenium
2. BeautifulSoup

In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
# for chrome
from selenium import webdriver

In [42]:
driver = webdriver.Chrome(executable_path='E:/chromedriver.exe')

# Startup the webdriver

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

#driver = webdriver.Chrome(executable_path='C:/Users/Admini/some problems/chromedriver.exe')

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Admini\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [4]:
url = 'https://www.amazon.com'
driver.get(url)

In [5]:
def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [6]:
url = get_url('fictional books')
print(url)

https://www.amazon.com/s?k=fictional+books&ref=nb_sb_noss_1


In [7]:
driver.get(url)

# Extract the collection

In [29]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [30]:
results = soup.find_all('div', {"data-component-type": "s-search-result"})

In [31]:
len(results)

48

# Prototype the record

In [11]:
item = results[0]

In [12]:
atag = item.h2.a

In [13]:
book_name = atag.text.strip()

In [14]:
url = 'https://www.amazon.com' + atag.get('href')

In [15]:
price_parent = item.find('span', 'a-price')

In [16]:
price = price_parent.find('span', 'a-offscreen').text

In [17]:
rating = item.i.text

In [18]:
author_div = item.find('div', 'a-row a-size-base a-color-secondary')

In [19]:
author = author_div.find('a', 'a-size-base a-link-normal').text.strip()

# Generalize the pattern

In [20]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # Book name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    # Author of book
    author_div = item.find('div', 'a-row a-size-base a-color-secondary')
    author = author_div.find('a', 'a-size-base a-link-normal').text.strip()
    
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    # rating
    rating = item.i.text
    
    result = (book_name, author, price, rating, url)
    
    return result

In [21]:
records = []
results = soup.find_all('div', {'data-component-type': "s-search-results"})

for item in results:
    records.append(extract_record(item))

# Error Handling

In [33]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # Book name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        # Author of book
        author_div = item.find('div', 'a-row a-size-base a-color-secondary')
        author = author_div.find('a', 'a-size-base a-link-normal').text.strip()
    except AttributeError:
        author = ''
        
    try:
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rating
        rating = item.i.text
    except AttributeError:
        rating = ''
    
    result = (book_name, author, price, rating, url)
    
    return result

In [34]:
records = []
results = soup.find_all('div', {"data-component-type": "s-search-result"})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [39]:
records[4]

('Salt to the Sea',
 'Ruta Sepetys',
 '$9.99',
 '4.7 out of 5 stars',
 'https://www.amazon.com/Salt-Sea-Ruta-Sepetys-ebook/dp/B00YM34WM8/ref=sr_1_6?dchild=1&keywords=fictional+books&qid=1617936390&sr=8-6')

# Getting the next page

In [ ]:
def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    #add page query placeholder
    url = template.format(search_term)
    
    # dd page query placeholder
    url += '&page{}'
    
    return url

# Putting it all together

In [5]:
import csv
from bs4 import BeautifulSoup
# for chrome
from selenium import webdriver

def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    #add page query placeholder
    url = template.format(search_term)
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    # Book name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        # Author of book
        author_div = item.find('div', 'a-row a-size-base a-color-secondary')
        author = author_div.find('a', 'a-size-base a-link-normal').text.strip()
    except AttributeError:
        author = ''
        
    try:
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rating
        rating = item.i.text
    except AttributeError:
        rating = ''
    
    result = (book_name, author, price, rating, url)
    
    return result

def main(search_term):
    """Run main Program routine"""
    # atartup the webdriver
    driver = webdriver.Chrome(executable_path='E:/chromedriver.exe')
    
    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        
        driver.get(url + '&page{}'.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {"data-component-type": "s-search-result"})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
        
    driver.close()
    
    # save data to csv file
    with open('bookdata.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Book Name', 'Author', 'Price', 'Rating', 'URL'])
        writer.writerows(records)

In [6]:
main('books')